In [2]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-15 23:34:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.51MB/s    in 0.2s    

2022-11-15 23:34:13 (5.51 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# create spark session

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,BooleanType,DateType
spark = SparkSession.builder.appName("big-data-challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
# Load file

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz" 
# get data from amazon S3 and  unzip extract files 

spark.sparkContext.addFile(url)

ebooks_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")

ebooks_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   35471030|R1C5OK9AND7PRP|B00AHK07V0|     175130663|Hunter's Moon (A ...|Digital_Ebook_Pur...|          5|            0|          0|   N|                Y|Shugak is like a ...|This is the most ...|2015-01-31 00:08:00|
|         US|   26579324|R333RNBQMPVUFT|B014085WTQ|     859232728|Flying

In [6]:

print ("The of records for the ebooks data set is:",ebooks_df.count())


The of records for the ebooks data set is: 12520722


In [7]:
# Drop null values, by default is any which means will drop any rows which has a NA
ebooks_df = ebooks_df.dropna(how='any')
ebooks_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   35471030|R1C5OK9AND7PRP|B00AHK07V0|     175130663|Hunter's Moon (A ...|Digital_Ebook_Pur...|          5|            0|          0|   N|                Y|Shugak is like a ...|This is the most ...|2015-01-31 00:08:00|
|         US|   26579324|R333RNBQMPVUFT|B014085WTQ|     859232728|Flying

In [8]:
# clean data by dropping NA and get count of data

print("The count after dropping NA ", ebooks_df.count())

The count after dropping NA  12518702


In [9]:
#drop duplicates 
ebooks_df = ebooks_df.dropDuplicates(["review_id","customer_id", "product_id"])

print("The count after dropping dropping Duplicates ", ebooks_df.count())

The count after dropping dropping Duplicates  12518702


In [10]:
#create dataframe that corresponds to customers table  with counts of records per user
customer_info_df = ebooks_df.groupby("customer_id").agg({"customer_id": "count"})
customer_info_df = customer_info_df.withColumnRenamed("count(customer_id)", "customer_count")
customer_info_df.show(10)



+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   30412198|            32|
|   10488134|            28|
|   19530671|           578|
|   16395903|             4|
|   14898270|             2|
|   25447005|             7|
|   11933163|             1|
|   26248773|           511|
|   30336397|             2|
|   41952424|            13|
+-----------+--------------+
only showing top 10 rows



In [11]:
##create dataframe that corresponds to columns in Products to load into data
product_df = ebooks_df.select(["product_id","product_title"])
product_df.show(10)
product_df.dtypes


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CGOSBTU|          Dark Space|
|B00WYJ2M24|Hearts of the Wes...|
|B00ILD4C8E|Can't Shake You (...|
|B00ILYTJS6|Lead: A Stage Div...|
|B00JI4ZSF6|Saint Odd: An Odd...|
|B00Q604NUG|Young, Restless A...|
|B00KLS30RQ|     Wife on the Run|
|B005CWUFAQ|Teaching the Tree...|
|B00FX6OYL2|Crock Pot Recipes...|
|B00JQQ1QQ6|Winds of Skilak: ...|
+----------+--------------------+
only showing top 10 rows



[('product_id', 'string'), ('product_title', 'string')]

In [12]:
from pyspark.sql.functions import *
#create dataframe that corresponds to columns in review id table to load into data
vine_df = ebooks_df.select(["review_id",col("star_rating").cast('int').alias("star_rating"),"helpful_votes","total_votes","vine"])
vine_df.show(10)
vine_df.dtypes


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000AOQ477IHO|          5|            1|          1|   N|
|R1000CY8CUFMZ9|          5|            1|          1|   N|
|R1000F05FWCLJ9|          1|            1|          5|   N|
|R1000KWQMMAXGC|          4|            0|          0|   N|
|R1000N261FFXXE|          5|            1|          2|   N|
|R1000N9U799D72|          5|            0|          0|   N|
|R1000QRIVWFX35|          5|            0|          0|   N|
|R10013PY0YSUUA|          5|            0|          0|   N|
|R10018GZO3GUSW|          4|            1|          1|   N|
|R1001O3QKO5WQR|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [13]:

#create dataframe that corresponds to columns in vine_table to load into data
review_id_df = ebooks_df.select(["review_id","customer_id","product_id","product_parent", 
                                 to_date(col("review_date"),"yyyy-mm-dd").alias("review_date")])
review_id_df.show(10)
review_id_df.dtypes


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1OHX24N3SF19Y|    3551713|B000UOJTUS|     979924066| 2015-01-31|
| R4YGPE8NN4GYX|   51592060|B000FBJGEK|     814935595| 2015-01-31|
| R1HNNEC53LM3A|   24774342|B013EJM84O|     720512082| 2015-01-31|
| RWJ0D9J0IB44E|   15231743|B012QYRI70|     910744117| 2015-01-31|
|R3PDPOO3M5B6M8|   34735923|B00CMEPMP0|     429592122| 2015-01-31|
| RESQGFFOZ53XB|   49311003|B00ERY9RUE|     722237307| 2015-01-31|
|R1NZBHLU2WS5V9|   38042627|B014AL0E96|     731783725| 2015-01-31|
|R1Y8PZ55IIKF1J|   47818107|B00YALTL28|     769015206| 2015-01-31|
|R19AQ5HOMURH7R|   47788531|B00VVQILD0|     127319812| 2015-01-31|
| R1O7272BWVJLO|   46781834|B00Y6QF0PE|     474654167| 2015-01-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [14]:
# Configure settings to connect Postgres
mode = "overwrite"  # use append here if adding data to the table
jdbc_url="jdbc:postgresql://gatechtest.cdie7a4an9rx.us-east-1.rds.amazonaws.com:5432/reviewdata"
config = {"user":"postgres", 
          "password": "Georgia!906", 
          "driver":"org.postgresql.Driver"}


In [15]:
#load data into customers table in AWS Postgres database
customer_info_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [16]:
customer_table_count = spark.read.jdbc(url=jdbc_url, table='customers', properties=config)
print("Load of customer table complete, number of records are:", customer_table_count.count())

Load of customer table complete, number of records are: 4030694


In [17]:
# Prints the number of records in each dataframe
print("Number of records for dataframe customer_info_df", customer_info_df.count())

Number of records for dataframe customer_info_df 4030694


In [18]:
 # load  data into review_id_table AWS Postgres database
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
#review how many records in the table


review_id_table_count = spark.read.jdbc(url=jdbc_url, table='review_id_table', properties=config)
print("Load of review_id_table table complete, total number of records are:", review_id_table_count.count())

Load of review_id_table table complete, total number of records are: 12518702


In [20]:
# Prints the number of records in each dataframe

print("Number of records for dataframe review_id_df", review_id_df.count())

Number of records for dataframe review_id_df 12518702


In [21]:
#load data into Products table in AWS Postgres database
#review how many records in the table 
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [22]:
products_table_count = spark.read.jdbc(url=jdbc_url, table='products',properties=config)
print("Load of products table complete, total number of records are:", products_table_count.count())

Load of products table complete, total number of records are: 12518702


In [23]:
# Prints the number of records in each dataframe

print("Number of records for dataframe products", product_df.count())

Number of records for dataframe products 12518702


In [24]:
#load data into vine_table in AWS Postgres Database
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [25]:
vine_table_count = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
print("Load of vine table complete, total number of records are:", vine_table_count.count())

Load of vine table complete, total number of records are: 12518702


In [26]:
# Prints the number of records in each dataframe

print("Number of records for dataframe vines df", vine_df.count())


Number of records for dataframe vines df 12518702
